In [17]:
import flask
from flask import request, jsonify
from flask_cors import CORS
from flask import render_template
import sqlite3
import json

In [18]:
app = flask.Flask(__name__)
app.config["DEBUG"] = True
CORS(app)

In [19]:
def dict_factory(cursor, row):
    d = {}
    for idx, col in enumerate(cursor.description):
        d[col[0]] = row[idx]
    return d

In [20]:
# -----If no URL route is found, return 404 not found error-----
@app.errorhandler(404)
def page_not_found(e):
    return "<h1>PIDX API Server - 404</h1><p> </p><p>The resource could not be found.</p>", 404

In [21]:
# -----A Route to return all of the Ticket Handling data-----
# Sample Call http://api.pidx.org:8080/api/v1/json/receive_tickets
@app.route('/api/v1/json/recieve_tickets', methods=['GET'])
def api_json_ticket_data():

    conn = sqlite3.connect('TicketTest.db')
    conn.row_factory = dict_factory
    cur = conn.cursor()

#   all_json_ticket_data = cur.execute('SELECT * FROM ticketData;').fetchall()
    all_json_ticket_data = cur.execute('ticketID, buyerParty, supplierParty, ticketCost FROM ticketData;').fetchall()
    conn.close()
    return jsonify({'result': [dict(row) for row in all_json_ticket_data]})

In [22]:
# -----A route to store ticket records in the ticketData table-----
# Sample Call http://api.pidx.org:8080/api/v1/json/submit_ticket/myuniquemessageID
@app.route('/api/v1/json/submit_ticket/<uuid>', methods=['GET', 'POST'])
def add_message(uuid):
    print('----PIDX Received POST Request---------------')

    content = request.json

    print('----Received POST Request---------------')
    print(content['ticketData'])
    print('-------------------')
    
    # open SQLite3 database
    conn = sqlite3.connect('TicketTest.db')
    cur = conn.cursor()
    
    # write new ticket record to the DB
    cur.execute('INSERT INTO ticketData (ticketID, buyerParty, supplierParty, ticketCost) VALUES (?, ?, ?, ?)',
      (content.get('ticketData').get('ticketID'), content.get('ticketData').get('buyerParty'), content.get('ticketData').get('supplierParty'),
       content.get('ticketData').get('ticketCost') ) )

    # commit and close DB
    conn.commit()
    conn.close()
    
    # return unique UUID sent from request
    return jsonify({"uuid":uuid})

In [23]:
# -----------------------------
# Start the server on LOCALHOST port 8080
# -----------------------------
if __name__ == '__main__':
      app.run(host='127.0.0.1', port=8080, debug=True)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


 * Running on http://127.0.0.1:8080/ (Press CTRL+C to quit)
 * Restarting with stat


SystemExit: 1

/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3334: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
